In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features.build_features import obtener_distribucion_consultas

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
# Carga la base de TrackCare
df_track = pd.read_csv("../data/processed/trackcare_procesada.csv")
df_track["prestacion"] = df_track["prestacion"].str.upper().str.strip()

# Deja solamente las consultas atendidas
df_track_atendidas = df_track.query("estadocita == 'Atendido'")

In [ ]:
# Obtiene resumen de consultas por anio
agrupacion_anual = ["ano", "especialidad_agrupada"]
distribucion_consultas_por_anio, consultas_por_paciente_por_anio = obtener_distribucion_consultas(
    df_track_atendidas, agrupacion_anual
)

# Obtiene resumen de consultas acumuladas en el periodo
agrupacion_acumulada = ["especialidad_agrupada"]
distribucion_consultas_acumuladas, consultas_por_paciente_acumuladas = (
    obtener_distribucion_consultas(df_track_atendidas, agrupacion_acumulada)
)

In [ ]:
# Define las glosas solamente de consultas (filtra procedimientos y miscaleneos)
GLOSAS_CONSULTAS = ["Consulta Repetida", "Consulta Nueva"]
consultas_validas = df_track.query(
    "estadocita == 'Atendido' and tipoatencion.isin(@GLOSAS_CONSULTAS)"
)
consultas_validas = consultas_validas.query("prestacion.str.contains('CONSULTA')")

# Define los profesionales medicos
PROFESIONALES_MEDICOS = ["Médico Cirujano", "Médico", "Psiquiatra"]

# Filtra por consultas medicas y profesionales medicas
consultas_medicas = consultas_validas.query("tipoprofesional.isin(@PROFESIONALES_MEDICOS)")

# Obtiene consultas no medicas
consultas_no_medicas = consultas_validas.query("~tipoprofesional.isin(@PROFESIONALES_MEDICOS)")

In [ ]:
# Obtiene la cantidad de consultas medicas
cantidad_consultas_medicas = consultas_medicas.groupby("ano").size()
cantidad_consultas_medicas.name = "n_consultas_medicas"

# Obtiene la cantidad de consultas no medicas por tipo de profesional
cantidad_consultas_no_medicas = (
    consultas_no_medicas.groupby(["ano", "tipoprofesional"])
    .size()
    .reset_index(1, name="n_consultas_no_medicas")
)

# Une la cantidad de consultas medicas y no medicas
resumen_consultas_medicas_y_no_medicas = cantidad_consultas_no_medicas.join(
    cantidad_consultas_medicas
)

# Obtiene la fraccion de cada no medicas de las medicas
resumen_consultas_medicas_y_no_medicas["proporcion_no_medicas"] = (
    resumen_consultas_medicas_y_no_medicas["n_consultas_no_medicas"]
    / resumen_consultas_medicas_y_no_medicas["n_consultas_medicas"]
) * 100

resumen_consultas_medicas_y_no_medicas = resumen_consultas_medicas_y_no_medicas.reset_index()

In [ ]:
sns.barplot(
    resumen_consultas_medicas_y_no_medicas,
    hue="tipoprofesional",
    x="ano",
    y="proporcion_no_medicas",
)

In [ ]:
resumen_consultas_medicas_y_no_medicas.sort_values(["tipoprofesional", "ano"])

In [ ]:
resumen_acumulado = resumen_consultas_medicas_y_no_medicas.groupby("tipoprofesional").sum()
resumen_acumulado["proporcion_no_medicas"] = (
    resumen_acumulado["n_consultas_no_medicas"] / resumen_acumulado["n_consultas_medicas"]
) * 100

In [ ]:
consultas_no_medicas.query("tipoprofesional == 'Enfermera (o)'")["prestacion"].value_counts()

In [ ]:
df_track[df_track["prestacion"].fillna("").str.contains("Taller")]